In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import ArrayType
import sys

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.27:7077") \
        .appName("Question_C5")\
        .getOrCreate()

spark_context = spark_session.sparkContext
sqlContext = SQLContext(spark_context)

In [2]:
df = spark_session.read.csv('/home/ubuntu/LDSA/lab2/gap_data.csv', header="true", inferSchema="true", multiLine="true", escape="\"")
dfAppendix = spark_session.read.csv('/home/ubuntu/LDSA/lab2/sicCodes.csv', header="true", inferSchema="true", multiLine="true", escape="\"")
df.createOrReplaceTempView("gap_data")

In [3]:
#df.select("SicCodes").show()

In [4]:
#df.printSchema()

In [5]:
#dfAppendix.printSchema()

In [6]:
#dfAppendix.show()

In [7]:
dfAppendix.select(array('Min','Max').alias("min_max")).show()

+--------+
| min_max|
+--------+
|  [1, 3]|
|  [5, 9]|
|[10, 33]|
|[35, 35]|
|[36, 39]|
|[41, 43]|
|[45, 47]|
|[49, 53]|
|[55, 56]|
|[58, 63]|
|[64, 66]|
|[68, 68]|
|[69, 75]|
|[77, 82]|
|[84, 84]|
|[85, 85]|
|[86, 88]|
|[90, 93]|
|[94, 96]|
|[97, 98]|
+--------+
only showing top 20 rows



In [ ]:
#dfAppendix.select('Min','Max')

In [ ]:
#merge_cols = udf(lambda arr: to_array(arr), ArrayType(IntegerType()))

In [ ]:
#dfAppendix.withColumn('Min_Max', merge_cols(dfAppendix.select('Min', 'Max'))).show()

In [ ]:
#Get_min_max_udf = udf(lambda y: y.split(',')[2], IntegerType())
#df_with_min_max = dfAppendix.withColumn("_c0",Get_min_max_udf("dfAppendix"))
#sum_cols = udf(lambda arr: sum(arr), IntegerType())

In [ ]:
#sqlContext.createDataFrame([(101, 1, 16, 8)], ['ID', 'A', 'B', 'C'])\
#.withColumn('Result', sum_cols(array('A', 'B', 'C'))).show()

In [8]:
df = df.withColumn('SicCodes', df.SicCodes.substr(1, 2))

In [9]:
df.drop('age').filter(df["SicCodes"] != 1)

DataFrame[EmployerName: string, Address: string, CompanyNumber: string, SicCodes: string, DiffMeanHourlyPercent: double, DiffMedianHourlyPercent: double, DiffMeanBonusPercent: double, DiffMedianBonusPercent: double, MaleBonusPercent: double, FemaleBonusPercent: double, MaleLowerQuartile: double, FemaleLowerQuartile: double, MaleLowerMiddleQuartile: double, FemaleLowerMiddleQuartile: double, MaleUpperMiddleQuartile: double, FemaleUpperMiddleQuartile: double, MaleTopQuartile: double, FemaleTopQuartile: double, CompanyLinkToGPGInfo: string, ResponsiblePerson: string, EmployerSize: string, CurrentName: string, SubmittedAfterTheDeadline
: string]

In [10]:
composite = df.join(dfAppendix,(df['SicCodes']<= dfAppendix['Max'])&(df['SicCodes']>= dfAppendix['Min']))\
.select("sector",'EmployerName','SicCodes','DiffMeanHourlyPercent')
composite.show()

+--------------------+--------------------+--------+---------------------+
|              sector|        EmployerName|SicCodes|DiffMeanHourlyPercent|
+--------------------+--------------------+--------+---------------------+
|           EDUCATION|"Bryanston School...|      85|                 18.0|
|WHOLESALE AND RET...|"RED BAND" CHEMIC...|      47|                  2.3|
|INFORMATION AND C...|         118 LIMITED|      61|                  1.7|
|ADMINISTRATIVE AN...|   123 EMPLOYEES LTD|      78|                 41.0|
|ARTS ENTERTAINMEN...|        1610 LIMITED|      93|                -22.0|
|ACCOMMODATION AND...|1879 EVENTS MANAG...|      56|                 13.4|
|ARTS ENTERTAINMEN...|1LIFE MANAGEMENT ...|      93|                 15.1|
|HUMAN HEALTH AND ...|  1ST HOME CARE LTD.|      86|                 15.0|
|ACCOMMODATION AND...| 1STOP HALAL LIMITED|      56|                 11.9|
|       MANUFACTURING|2 AGRICULTURE LIM...|      14|                 13.4|
|       MANUFACTURING|2 S

In [ ]:
composite.createOrReplaceTempView("composite")

In [ ]:
sqlContext.sql("SELECT sector, AVG(DiffMeanHourlyPercent) FROM composite GROUP BY sector ORDER BY AVG(DiffMeanHourlyPercent) ASC").show()